# Importing Libraries 

In [56]:
import numpy as np
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import pickle
import re
from scipy import spatial
import nltk.data



# Downloading PreTrained BERT

In [57]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [58]:
sentence_splitter= nltk.data.load('tokenizers/punkt/english.pickle')

In [63]:
alpha=0.7

# Loading Data

In [60]:
#For Statutes
directory= r"C:\Users\HP\Desktop\Study\COL764\Project\AILA_2019_dataset\Object_statutes"
# #For Precedents
# directory= r"C:\Users\HP\Desktop\Study\COL764\Project\AILA_2019_dataset\Object_casedocs"


# Making a dictionary of Query Vectors

In [61]:
lawbert_query={}
f=open('AILA_2019_dataset/Query_doc.txt','r')

query_file = f.read()

temp_queries = query_file.split("\n")


for query in temp_queries:
    doc_curr = query.split("||")[1]
    tokenized=torch.tensor([tokenizer.encode(doc_curr, add_special_tokens=True,padding=True, 
                                            truncation=True, max_length=512)])

    #For CLS
    with torch.no_grad():
        lawbert_query[query.split("||")[0]]=model(tokenized)['last_hidden_state'][0][0].detach().numpy()
#     #For Mean
#     with torch.no_grad():
#         lawbert_query[query.split("||")[0]]=model(tokenized)['last_hidden_state'].mean(1)[0].detach().numpy()

# Making Document Vectors (Ranking tasks)

In [ ]:
lawbert_dic={}
for subdir, dirs, files in os.walk(directory):
    for file in files:
        with open(subdir+'/'+file,mode="r", encoding = "utf8", errors="ignore") as f:
            doc_curr=f.read()
            tokenized=torch.tensor([tokenizer.encode(doc_curr, add_special_tokens=True,padding=True, 
                                                    truncation=True, max_length=512)])

            #For CLS
            with torch.no_grad():
                lawbert_dic[str(file[:-4])]=model(tokenized)['last_hidden_state'][0][0].detach().numpy()
#             #For SEP
#             with torch.no_grad():
#                 lawbert_dic[str(file[:-4])]=model(tokenized)['last_hidden_state'][0][-1].detach().numpy()
#             #For Mean
#             with torch.no_grad():
#                 lawbert_dic[str(file[:-4])]=model(tokenized)['last_hidden_state'].mean(1)[0].detach().numpy()

# Generating results (Ranking tasks)

In [ ]:
out_dict = {}

for query in lawbert_query.keys():
    scores=[]
    query_vec=lawbert_query[query]
    for doc in lawbert_dic.keys():
        doc_vec=lawbert_dic[doc]
        sim_score=spatial.distance.cosine(query_vec,doc_vec)
        scores.append((doc,sim_score))
    scores=sorted(scores, key = lambda x: x[1],reverse=True)
    out_dict[query]=scores

In [ ]:
with open('lawbert.pickle', 'wb') as handle:
    pickle.dump(out_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Generating Results (ReRanking) 

In [62]:
curr = pd.read_pickle(r'stfidf.pickle')
top_100={}
for q_id in curr.keys():
    top_100[q_id]=curr[q_id][:100]

In [67]:
# For tfidf and BM25 only
temp={}
for i in range(len(curr)):
    q_id='AILA_Q'+str(i+1)
    temp[q_id]=top_100[i]
top_100=temp

In [ ]:
out_dict = {}

for query in lawbert_query.keys():
    scores=[]
    query_vec=lawbert_query[query]
    for doc in top_100[query]:
        doc_id=doc[0]
        doc_score=doc[1]
        d=open(directory+'/'+doc_id+'.txt',mode="r", encoding = "utf8", errors="ignore")
        doc_curr=d.read()
#         if len(tokenizer.encode(doc_curr))>512:
#            print(len(tokenizer.encode(doc_curr)))
        sentences=sentence_splitter.tokenize(doc_curr)
        sim_scores=[]
        for sentence in sentences:
            tokenized=torch.tensor([tokenizer.encode(sentence, add_special_tokens=True,padding=True, 
                                                    truncation=True, max_length=512)])
            with torch.no_grad():
                doc_vec=model(tokenized)['last_hidden_state'][0][0].detach().numpy()
            sim_scores.append(spatial.distance.cosine(query_vec,doc_vec))
        sim_scores.sort(reverse=True)
#         print(sim_scores)
        sim_score=0
        if len(sim_scores)<10:
            for score in sim_scores:
                sim_score+=score
            sim_score/=len(sim_scores)
        else:
            for i in range(10):
                sim_score+=sim_scores[i]
            sim_score/=10
        sim_score=alpha*doc_score+(1-alpha)*sim_score
        scores.append((doc_id,sim_score))
    scores=sorted(scores, key = lambda x: x[1],reverse=True)
    out_dict[query]=scores
    

[1.1184108555316925, 1.1054635271430016, 1.0712312534451485, 1.0707828179001808, 1.060342200100422, 1.0602583847939968, 1.0563735887408257, 1.0540457554161549, 1.0505812168121338, 1.050067212432623, 1.050067212432623, 1.050067212432623, 1.044061116874218, 1.0411015823483467, 1.0378885976970196, 1.0355700813233852, 1.034432303160429, 1.0298351421952248, 1.0247679613530636, 1.0241369809955359, 1.0155700352042913, 0.9247857928276062, 0.8523828685283661, 0.7727441936731339, 0.7727441936731339, 0.7529559135437012, 0.7253116965293884, 0.7253116965293884, 0.7253116965293884, 0.7253116965293884, 0.7253116965293884, 0.7253116965293884, 0.7253116965293884, 0.7134663164615631, 0.7134663164615631, 0.7134663164615631, 0.7134663164615631, 0.7134663164615631, 0.7134663164615631, 0.7067961394786835, 0.7002239525318146, 0.6933042705059052, 0.6933042705059052, 0.6933042705059052, 0.6842523515224457, 0.6798601150512695, 0.6701314151287079, 0.6611308753490448, 0.6436260640621185, 0.6395487487316132, 0.637

[1.0589083582162857, 1.0460161603987217, 1.0366129502654076, 1.0247540529817343, 1.0223503168672323, 1.017402108758688, 0.6280123591423035, 0.6280123591423035, 0.6280123591423035, 0.6280123591423035, 0.5882432758808136, 0.5191789269447327]
[1.0702301785349846, 1.0682606995105743, 1.0576333031058311, 1.0570933520793915, 1.05704702809453, 1.0504628382623196, 1.0468152686953545, 1.0457839332520962, 1.035817064344883, 1.033990416675806, 0.660992830991745, 0.5510364770889282, 0.4566660523414612]
[1.0585653819143772, 1.0559710636734962, 1.0457253567874432, 1.0402055233716965, 1.0286101587116718, 1.025893047451973, 1.0172785371541977, 0.9999462414452864, 0.7030937671661377, 0.7030937671661377, 0.7030937671661377, 0.7030937671661377, 0.7030937671661377, 0.7030937671661377, 0.7030937671661377, 0.7030937671661377, 0.5499091148376465, 0.5499091148376465, 0.5499091148376465, 0.5280634462833405, 0.5241589844226837, 0.5241589844226837, 0.4439052939414978, 0.410334050655365, 0.39345622062683105, 0.38

[1.0607331842184067]
[1.0643815025687218, 1.0483591109514236, 1.0471996665000916, 1.0435979887843132, 1.0387003421783447, 1.0387003421783447, 1.0387003421783447, 1.0387003421783447, 1.0387003421783447, 1.0329398661851883, 0.6614440083503723, 0.6255808770656586, 0.6248176395893097, 0.610742598772049, 0.6085544526576996, 0.5703592598438263, 0.4828200340270996, 0.4608681797981262, 0.46075665950775146, 0.4567108154296875]
[1.0593226365745068, 1.058909498155117, 1.0576705485582352, 1.052136093378067, 1.052136093378067, 1.0521015785634518, 1.0438792817294598, 1.0422886461019516, 1.0388242900371552, 1.0388242900371552, 1.0375118404626846, 1.0375118404626846, 1.0323117226362228, 1.0276548620313406, 1.0276548620313406, 1.0276548620313406, 1.025855703279376, 0.6032500267028809, 0.6032500267028809, 0.6032500267028809, 0.5593335926532745, 0.5593335926532745, 0.5593335926532745, 0.5593335926532745, 0.5593335926532745, 0.5593335926532745, 0.5593335926532745, 0.5593335926532745, 0.5593335926532745, 0

[1.0597101114690304, 1.0502256527543068]
[1.081106722354889, 1.0678850188851357, 1.059958841651678, 1.0324789322912693, 1.026437722146511, 0.6593732237815857]
[1.0492031387984753, 1.0387003421783447, 1.0387003421783447, 1.0387003421783447, 1.036131776869297, 1.0359295010566711, 1.0349169559776783, 0.5861360430717468, 0.5660632848739624, 0.5660632848739624, 0.5444646179676056]
[1.077147625386715, 1.0666775703430176, 1.0657657757401466, 1.0654242783784866, 1.0534206442534924, 1.0511272624135017, 1.050869733095169, 1.050843209028244, 1.0503096990287304, 1.0500374510884285, 1.049192551523447, 1.048664391040802, 1.047518029808998, 1.0468358285725117, 1.0456856191158295, 1.0455463267862797, 1.0450024753808975, 1.0450024753808975, 1.0441361404955387, 1.0435353629291058, 1.0434071905910969, 1.0419029667973518, 1.038593202829361, 1.0331569947302341, 1.032847784459591, 1.0322731137275696, 1.0082110837101936, 1.005471047013998, 0.621587723493576, 0.6168064773082733, 0.6083508729934692, 0.60835087

[1.0987195447087288, 1.075694002211094, 1.0679020136594772, 1.0679020136594772, 1.0679020136594772, 1.0679020136594772, 1.0679020136594772, 1.05785720795393, 1.0567394904792309, 1.0563780777156353, 0.6424472630023956, 0.6350334882736206, 0.6131523549556732, 0.6046742796897888, 0.5970501899719238, 0.5378887355327606, 0.4866313934326172, 0.45291048288345337, 0.4415237307548523, 0.4271232485771179]
[1.079059176146984, 1.059687566012144, 1.05697862803936, 0.639863520860672, 0.5976989269256592]
[1.058211974799633, 0.4645037055015564]
[1.0676163882017136, 1.067422829568386, 0.5619872510433197, 0.5511001348495483, 0.2331010103225708]
[1.0856736227869987, 1.0832432061433792, 1.0798900425434113, 1.0528245083987713, 1.0444021597504616, 0.566390335559845, 0.46605587005615234]
[1.096721664071083, 1.093265362083912, 1.0918462127447128, 1.0851028636097908, 1.0809723809361458, 1.0804284289479256, 1.0799198001623154, 1.0788792222738266, 1.0732482820749283, 1.0712457969784737, 1.0711480528116226, 1.066

...

[1.0790295898914337, 1.0641473308205605, 1.0595570467412472, 1.0493270233273506, 1.0468466393649578, 1.0468466393649578, 1.0468466393649578, 1.0468466393649578, 1.0468466393649578, 1.0468466393649578, 1.0423422530293465, 1.0364405252039433, 1.0364405252039433, 1.0337365418672562, 1.031395062804222, 1.0268294662237167, 1.0262239780277014, 1.0262239780277014, 1.0262239780277014, 1.0262239780277014, 1.0262239780277014, 1.0262239780277014, 0.710409551858902, 0.710409551858902, 0.6672715246677399, 0.6505440771579742, 0.6352300047874451, 0.6352300047874451, 0.6352300047874451, 0.6352300047874451, 0.6352300047874451, 0.6352300047874451, 0.6080256402492523, 0.5948657691478729, 0.5930219888687134, 0.5930219888687134, 0.5930219888687134, 0.5930219888687134, 0.5930219888687134, 0.5655092597007751, 0.5308892130851746, 0.5308892130851746, 0.5302684605121613, 0.5021459460258484, 0.4506279230117798, 0.4447196125984192, 0.44461774826049805, 0.37614530324935913, 0.24190443754196167]
[1.102316752076149,

[1.0473813936114311, 1.043535690754652, 0.8217960298061371]
[1.0919207632541656, 1.0206133369356394, 1.0206133369356394, 0.6417071223258972, 0.6417071223258972, 0.5988795459270477, 0.5757204592227936, 0.5757204592227936, 0.5660816729068756, 0.5362585186958313, 0.5170776844024658, 0.5170776844024658, 0.4907371997833252, 0.4069456458091736, 0.4069456458091736, 0.2605116367340088]
[1.080253191292286, 1.0702457651495934, 1.0658454596996307, 1.065367802977562, 1.0636005103588104, 1.0557758323848248, 1.0554386302828789, 1.047435935586691, 1.0432142317295074, 1.0416854284703732, 1.0416854284703732, 1.0386346466839314, 1.0239888485521078, 0.9557949863374233, 0.7959011495113373, 0.735355406999588, 0.6721812188625336, 0.6695531010627747, 0.6473881900310516, 0.6412329971790314, 0.6108543276786804, 0.6100189685821533, 0.5798971056938171, 0.5766380727291107, 0.5733133554458618, 0.5692031979560852, 0.5599513351917267, 0.5535079836845398, 0.5459005832672119, 0.5300677120685577, 0.5257223546504974, 0.

[1.0628549084067345, 1.0628549084067345, 1.0514837205410004, 1.050802905112505, 1.0466706976294518, 1.0447620563209057, 1.0368518494069576, 1.0360232815146446, 1.0358554758131504, 1.0299495309591293, 0.7924792915582657, 0.7307191789150238, 0.6583147346973419, 0.6583147346973419, 0.5916422605514526, 0.5789394974708557, 0.5718630850315094, 0.4898049831390381, 0.30195146799087524]
[1.0846359729766846, 1.0733452141284943, 0.6548055708408356, 0.49645519256591797, 0.4882705807685852]
[1.0417498722672462]
[1.0718703716993332]
[1.080643653869629, 1.0698207020759583, 1.0667729079723358, 1.0578255541622639, 1.0491696745157242, 1.041297759860754, 1.0351084396243095, 0.5027620196342468, 0.48043084144592285, 0.3248988389968872]
[1.0727987959980965, 1.0682595744729042]
[1.0580170303583145]
[1.0680066123604774, 1.0657314732670784, 1.0635848566889763, 1.0600386708974838, 1.0596865229308605, 1.0592780113220215, 1.0584092922508717, 1.0550964139401913, 1.0544084571301937, 1.0511903874576092, 1.0498195774

[1.0631296187639236, 1.057614777237177, 0.4559221863746643]
[1.0688744932413101, 1.0453294701874256, 0.5756458640098572, 0.5687243044376373, 0.5458264052867889, 0.5294358432292938, 0.5231654047966003, 0.5160064995288849, 0.5145611763000488, 0.4287566542625427]
[1.0784917771816254, 1.0719963312149048, 1.065688855946064, 1.065688855946064, 1.0646546259522438, 1.062683418393135, 1.0598732978105545, 1.0459401905536652, 0.40470486879348755, 0.40470486879348755, 0.40470486879348755]
[1.0868732780218124, 1.0846797674894333, 1.0737182050943375, 1.050027210265398, 1.049262911081314, 0.5174460709095001, 0.45323944091796875]
[1.059979785233736, 1.055621910840273, 1.0541318356990814, 1.0526421442627907, 1.0525629967451096, 1.0501130633056164, 0.6514284610748291]
[1.1085296049714088, 1.0807288065552711, 1.0805407091975212, 1.0749137103557587, 1.0710267946124077, 1.067861944437027, 1.0658109858632088, 1.065455213189125, 1.064672589302063, 1.0643108636140823, 1.06358702480793, 1.0614652000367641, 1.0

[1.0728404521942139, 1.0716197043657303, 1.0679487213492393, 1.0676367431879044, 1.0669242292642593, 1.0633300319314003, 1.0617641620337963, 1.0614815428853035, 1.0582955814898014, 1.0578499548137188, 1.055536463856697, 1.053578071296215, 1.0465511083602905, 1.044561106711626, 1.0445554964244366, 1.0425339713692665, 0.8989254906773567, 0.8989254906773567, 0.6605839431285858, 0.5652381777763367, 0.5416236817836761, 0.5347496271133423, 0.5334495306015015]
[1.0554854311048985]
[1.073585718870163, 1.0578145310282707, 1.0556321144104004, 1.047315590083599, 1.0343948639929295, 0.6046716868877411, 0.6000232398509979, 0.5822655856609344, 0.5737686455249786, 0.5737686455249786, 0.4498111605644226, 0.389321506023407]
[1.0826905369758606, 1.0573068484663963, 1.0544207505881786, 1.0481685027480125, 0.6249112784862518]


In [ ]:
with open('bert_tfidf_statutes.pickle', 'wb') as handle:
    pickle.dump(out_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
out_dict